In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as DT
import matplotlib.dates

In [50]:
#import driver data from csv
driver = pd.read_csv('driver_ids.csv')

driver['driver_onboard_date'] = pd.to_datetime(driver['driver_onboard_date'])
#check driver data basic summary 
driver.describe()


,driver_id,driver_onboard_date
count,937,937
unique,937,49
top,a85adc80be31a10c23b7aaf380e39173,2016-04-05 00:00:00
freq,1,36
first,NaN,2016-03-28 00:00:00
last,NaN,2016-05-15 00:00:00


In [51]:
#no missing values
driver.isnull().sum()

driver_id              0
driver_onboard_date    0
dtype: int64

In [52]:
#look at driver onboard date distribution

#date_strings = [date.replace(' 00:00:00', '') for date in driver['driver_onboard_date']]
#date_objects = [DT.datetime.strptime(date, '%Y-%m-%d') for date in date_strings]
date_objects = driver['driver_onboard_date']

#date_strings = set(date_strings)
#date_obj_set = [DT.datetime.strptime(date, '%Y-%m-%d') for date in date_strings]
#date_obj_set.sort()
days = (max(driver['driver_onboard_date']) - min(driver['driver_onboard_date'])).days


plt.rcParams["figure.figsize"] = [50,20]
plt.hist(date_objects, bins=days+1)

plt.show()

TypeError: invalid type promotion

The data sample contains drivers onboarded between 2016-03-28 to 2016-05-15, the distribution is fairly uniform with weekly cyclic pattern 

In [14]:
#import ride data and check summary
ride = pd.read_csv('ride_ids.csv')
ride.describe()

,ride_distance,ride_duration,ride_prime_time
count,193502.000000,193502.000000,193502.000000
mean,6955.218266,858.966099,17.305893
std,8929.444606,571.375818,30.825800
min,-2.000000,2.000000,0.000000
25%,2459.000000,491.000000,0.000000
50%,4015.000000,727.000000,0.000000
75%,7193.000000,1069.000000,25.000000
max,724679.000000,28204.000000,500.000000


There are records showing negative ride distance, it's probably due to data tracking error which need to be removed

In [24]:
#check missing value
ride.isnull().sum()

driver_id          0
ride_id            0
ride_distance      0
ride_duration      0
ride_prime_time    0
dtype: int64

In [25]:
#remove the records with negative ride distince 
ride = ride [ride['ride_distance']>0]
ride.describe()

,ride_distance,ride_duration,ride_prime_time
count,193315.000000,193315.000000,193315.000000
mean,6961.946290,859.633965,17.311254
std,8931.140532,571.122920,30.832015
min,1.000000,2.000000,0.000000
25%,2463.000000,492.000000,0.000000
50%,4020.000000,727.000000,0.000000
75%,7197.500000,1069.000000,25.000000
max,724679.000000,28204.000000,500.000000


In [26]:
#import ride ts data
ride_ts = pd.read_csv('ride_timestamps.csv')
ride_ts.describe()

,ride_id,event,timestamp
count,970405,970405,970404
unique,194081,5,865826
top,51b44538f52ef231421b782e947b0b67,arrived_at,2016-04-30 22:09:15
freq,5,194081,7


In [29]:
#missing value?
ride_ts.isnull().sum()

ride_id      0
event        0
timestamp    1
dtype: int64

In [42]:
#what's the missing value
ride_ts[ride_ts.isnull().any(axis=1)]

,ride_id,event,timestamp
434222,72f0fa0bd86800e9da5c4dced32c8735,arrived_at,NaN


Since there is only one missing timestamp, we can do an imptation later to complete the data set

In [30]:
#data coherency check: to see if every ride has all 5 events
events_per_ride = ride_ts.groupby(['ride_id']).nunique()
events_per_ride.describe()

,ride_id,event,timestamp
count,194081.0,194081.0,194081.000000
mean,1.0,5.0,4.878391
std,0.0,0.0,0.329926
min,1.0,5.0,3.000000
25%,1.0,5.0,5.000000
50%,1.0,5.0,5.000000
75%,1.0,5.0,5.000000
max,1.0,5.0,5.000000


Event type is coherent, all rides in here have 5 unique events and no duplication in ride id

In [45]:
#pivot ride_ts table to make it convenient for future merging with ride data
ride_event = ride_ts.pivot(index='ride_id', columns='event', values='timestamp' )
ride_event.head()

event,accepted_at,arrived_at,dropped_off_at,picked_up_at,requested_at
ride_id,,,,,
00003037a262d9ee40e61b5c0718f7f0,2016-06-13 09:39:51,2016-06-13 09:44:31,2016-06-13 10:03:05,2016-06-13 09:44:33,2016-06-13 09:39:19
00005eae40882760d675da5effb89ae3,2016-05-14 05:23:25,2016-05-14 05:26:17,2016-05-14 05:34:17,2016-05-14 05:26:18,2016-05-14 05:23:21
000061d42cf29f73b591041d9a1b2973,2016-05-16 15:43:14,2016-05-16 15:47:29,2016-05-16 15:54:18,2016-05-16 15:47:32,2016-05-16 15:43:09
00006efeb0d5e3ccad7d921ddeee9900,2016-05-11 19:29:43,2016-05-11 19:35:11,2016-05-11 19:40:47,2016-05-11 19:35:15,2016-05-11 19:29:36
0000d9b24d8ccdd991b76258e616fa01,2016-04-26 18:12:15,2016-04-26 18:16:52,2016-04-26 18:24:14,2016-04-26 18:16:55,2016-04-26 18:11:38


In [49]:
ride_event['requested_at'] = pd.to_datetime(ride_event['requested_at'])
ride_event['accepted_at'] = pd.to_datetime(ride_event['accepted_at'])
ride_event['arrived_at'] = pd.to_datetime(ride_event['arrived_at'])
ride_event['picked_up_at'] = pd.to_datetime(ride_event['picked_up_at'])
ride_event['dropped_off_at'] = pd.to_datetime(ride_event['dropped_off_at'])
ride_event.describe()

event,accepted_at,arrived_at,dropped_off_at,picked_up_at,requested_at
count,194081,194080,194081,194081,194081
unique,190426,190526,190468,190524,190434
top,2016-06-02 16:00:22,2016-05-28 00:40:53,2016-06-03 18:59:26,2016-06-03 21:24:04,2016-05-06 20:39:08
freq,3,4,4,4,4
first,2016-03-28 05:48:23,2016-03-28 06:02:36,2016-03-28 06:37:51,2016-03-28 06:02:39,2016-03-28 05:48:18
last,2016-06-26 23:57:49,2016-06-27 00:02:03,2016-06-27 00:50:50,2016-06-27 00:02:06,2016-06-26 23:57:45


In [13]:
#use drop off ts as the completion time
ride_cmplt_ts = ride_ts[ride_ts['event']=='dropped_off_at']
ride_cmplt_ts.describe(

,ride_id,event,timestamp
count,194081,194081,194081
unique,194081,1,190468
top,51b44538f52ef231421b782e947b0b67,dropped_off_at,2016-06-03 18:59:26
freq,1,194081,4


In [25]:
ride = pd.read_csv('ride_ids.csv')

driver_ride = pd.merge(driver, ride, how= 'outer', left_on='driver_id', right_on='driver_id')

#driver_ride.isnull().sum()


driver_id                 0
driver_onboard_date    7611
ride_id                  83
ride_distance            83
ride_duration            83
ride_prime_time          83
dtype: int64